In [ ]:
!pip3 install -q langchain faiss-cpu pypdf openai tiktoken langchain-openai langchain_chroma

In [2]:
import os
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import openai
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain.chains.base import Chain
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.chains import RetrievalQA, load_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
import PyPDF2


In [5]:
template = """Answer the question based only on the following context. If you cannot answer the question with the context, please respond with 'I don't know':

### CONTEXT
{context}

### QUESTION
Question: {question}
"""


In [6]:
def read_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        num_pages = reader.pages
        text = ''
        for page_num in num_pages:
            text += page_num.extract_text()
    return text

def chunk(document):
  text_splitter = RecursiveCharacterTextSplitter(
    separators=[".", "\n"],
    chunk_size=1024,
    chunk_overlap=200,
    length_function=len,
  )
  splits = text_splitter.create_documents([document])
  return splits

docs = read_pdf("./BIM.pdf")
splits = chunk(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(),persist_directory="./chroma_db")

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
LLM_NAME="gpt-3.5-turbo"
from langchain.prompts import ChatPromptTemplate

llm = ChatOpenAI(temperature=0)
prompt = ChatPromptTemplate.from_template(template)
base_retriever = vectorstore.as_retriever()